In [22]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data #imported the required modules
mnist=input_data.read_data_sets("/tmp/data/",one_hot=True)#this loads the mnist data into tensorflow in one_hot encoding which encode categorial data into vector of number

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


one hot encoding converts data of categorial variables into binary vectors. ONLY 1 interger is marked 1 and rest are zero. eg: in 0-9 encoding '1' is encoded as  "1=[0 1 0 0 0 0 0 0 0 0]"

In [0]:
n_classes=10 #these are number of class labels
batch_size=128 #it defines how many dimensions you will feed at a time 
x = tf.placeholder('float', [None, 784]) #this is the input placeholder
y = tf.placeholder('float')# this is output placeholder, first dimension is non as batch defined it already

In [0]:
#conv2d with input data and weight is used for convolution of input, window will move 1 pixel at a time 
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
#for finding the max in a window . Currently 2*2 window moves 2 pixels at a time
def maxpool2d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


You will padding equal to same which ensures that while performing the convolution operations, the boundary pixels of the image are not left out, so padding equal to same will basically adds zeros at the boundaries of the input and allow the convolution filter to access the boundary pixels as well.

In [0]:
#function for cnn model implementation
def convolutional_neural_network(x):
    weights = {
                'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),#1st hidden layer, weight with 5*5 convulation,1 input channel,32 output channels
               'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),#2nd hidden layer, weight with 5*5 convlation, 32 input channel, 64 output channels
               'W_fc':tf.Variable(tf.random_normal([7*7*64,1024])),# fully connected layer with 7*7*64 input and 1024 outputs
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}# output layer with 10 output
    #these have dimension according to each layer output channels
    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}
    #reshape our input data to a 4d tensor
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    #convulation layer 1
    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])#rectified linear function to implement (input*weights)+ biases
    conv1 = maxpool2d(conv1)#activation of layer and maxpool after convulation
    #convulation layer 2
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])#rectified linear function to implement (input*weights)+ biases
    conv2 = maxpool2d(conv2)#activation of layer and maxpool after convulation
    #fully connected layer
    fc = tf.reshape(conv2,[-1, 7*7*64])#reshaping conv2 to fit fc layer
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
   

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output


In [0]:
#funtion for traning and testing the data
def train_neural_network(x):
  prediction = convolutional_neural_network(x)#prediction of output using our model
   
  cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )#comparing our output with actual output and finding the cost value
  
  optimizer = tf.train.AdamOptimizer().minimize(cost)  #using Adam optimizer(a famous deep learning optimization algo) to minimise the cost value
    
    #these are no. of cycle where each cycle->feed forward+backpropogation
      epochs1 = 10
    with tf.Session() as sess:   #this starts our tensorflow session
        
        sess.run(tf.initialize_all_variables())#intialize all the variable we declared above in our abstract

        for epoch in range(epochs1):
           #intialized as calculated in each interation
           epoch_loss = 0
          #dividing the dimension fed batchwise
            for _ in range(int(mnist.train.num_examples/batch_size)):
                #intialized for every batch size data
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                #cost is provided to optimizer and then cost is added to loss
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

          #printed to keep track of on cycle completion and cost after that 
          print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
        
        #check correctness of our predicted output
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        #calculates accuracy by finding the mean over 'correct' values
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
      #print the accuaracy 
      print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
#funtion call for training the neural network model by passing data as argument
train_neural_network(x)
